## Import library

In [400]:
# importing required libraries
import os, sys
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')
import data_load


from sklearn.metrics import log_loss

import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit
import evaluation as evl

from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Inputs
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Data viz
from mlens.visualization import corr_X_y, corrmat

# Model evaluation
from mlens.metrics import make_scorer
from mlens.model_selection import Evaluator

# Ensemble
from mlens.ensemble import SuperLearner

ccccccfrom sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from mlens.ensemble import SuperLearner

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [401]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


## Load data

In [1]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-1000.test.npz"

In [2]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"] 

In [3]:
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(10)
data = pd.read_csv(DATA_PATH, header = None, names = col)

NameError: name 'pd' is not defined

In [670]:
data.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,24.534220,19.525653,21.221590,24.830999,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,22.981254,26.063482,21.354089,24.837223,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,16.006497,23.937630,18.040368,24.668592,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,24.243270,23.527574,25.759277,25.024775,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,38.693296,25.436166,38.671451,25.431081,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


## Influence function

In [671]:
## Fit treatment
model_t = LogisticRegression()
# model_t = RandomForestClassifier(n_estimators=10)
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

data['predicted_y'] = model_y.predict(data[features])

## Compute propensity score
data['predicted_t'] = model_t.predict(data[cov])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']



    

### Performance
print("------------- Performance in treatment model --------------------")
print("F1 score: ", f1_score(data['t'], data['predicted_t'], average="macro"))
print("Precision: ", precision_score(data['t'], data['predicted_t'], average="macro"))
print("Recall: ",recall_score(data['t'], data['predicted_t'], average="macro")) 
print("\n")

# mean squared error
mse = np.sum((data['yf'] - data['predicted_y'].values)**2)
rmse = np.sqrt(mse/len(data))

print("------------- Performance in treatment model --------------------")
print("RMSE: ", rmse)

------------- Performance in treatment model --------------------
F1 score:  0.5141053747380087
Precision:  0.7242647058823529
Recall:  0.531037012495267


------------- Performance in treatment model --------------------
RMSE:  0.02494217479551891


In [672]:
data.p1

0      0.283647
1      0.067996
2      0.107668
3      0.129877
4      0.092954
5      0.258861
6      0.372819
7      0.261353
8      0.310520
9      0.106283
10     0.176722
11     0.134827
12     0.114987
13     0.452615
14     0.181261
15     0.106649
16     0.113266
17     0.254370
18     0.384934
19     0.117634
20     0.197417
21     0.237294
22     0.193818
23     0.107462
24     0.142221
25     0.202854
26     0.148585
27     0.209355
28     0.338374
29     0.174754
30     0.317762
31     0.096490
32     0.122482
33     0.074616
34     0.174196
35     0.181761
36     0.267310
37     0.123228
38     0.251159
39     0.331086
40     0.151520
41     0.250280
42     0.261577
43     0.397013
44     0.295051
45     0.231194
46     0.263441
47     0.095756
48     0.222867
49     0.314086
50     0.216566
51     0.357750
52     0.275049
53     0.371817
54     0.211170
55     0.138219
56     0.118571
57     0.247610
58     0.283350
59     0.114796
60     0.118084
61     0.160024
62     0

In [673]:
# data['ips_weight'] = (1 / len(data)) * (
#     data['t'] / data['p1'] +
#     (1 - data[treatment]) / (1 - data['p1']))

# # Calculating the effect
# data['d_y'] = (data['ips_weight'] *
#                      data['t'] *
#                      data[outcome])
# data['dbar_y'] = (data['ips_weight'] *
#                         (1 - data['t']) *
#                         data[outcome])
# est = data['d_y'].sum() - data['dbar_y'].sum()
# est

In [674]:
## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

In [675]:
delta = 2.5
## Compute incremental score 
data['q1'] = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
data['q0'] = 1 - data['q1']

In [676]:
simple_estimator = data['q1']*data['cf1'] + data['q0']*data['cf0']

In [677]:
# Y - mu(X,A)
data['residual'] = data[outcome] - data['predicted_y']
data['residual_cf0'] = data[outcome] - data['cf0']
data['residual_cf1'] = data[outcome] - data['cf1']

data['w0'] = [0 if data.loc[i,'predicted_t'] == 1 else 1/data.loc[i, 'p0'] for i in range(len(data))]
data['w1'] = [0 if data.loc[i,'predicted_t'] == 0 else 1/data.loc[i, 'p1'] for i in range(len(data))]

data['phi_0'] = data['w0'] * data['residual_cf0'] + data['cf0'] 
data['phi_1'] = data['w1'] * data['residual_cf1'] + data['cf1']

print(data[['phi_0', 'phi_1']].head())

data['ips_weight'] = (
    data[treatment] / data['p1'] +
    (1 - data[treatment]) / (1 - data['p1']))

data['phi_0'] = data['ips_weight'] * data['residual_cf0'] + data['cf0'] 
data['phi_1'] = data['ips_weight'] * data['residual_cf1'] + data['cf1']

print(data[['phi_0', 'phi_1']].head())


avg_phi = -data['q1']*data['phi_1'] + data['q0']*data['phi_0']

# avg_phi = data['q1']*data['phi_1'] + data['q0']*data['phi_0']

diff_cf = data['cf1'] - data['cf0']
diff_ps = data[treatment] - data['p1']
modi_ps = (delta*data['p1'] + data['p0'])**2

avg_diff = (delta*diff_cf*diff_ps) / modi_ps

       phi_0      phi_1
0  25.591799  24.537406
1  22.980897  26.206352
2  16.006722  20.796127
3  24.242936  23.672203
4  38.693657  28.780193
       phi_0      phi_1
0  31.279684  24.526176
1  22.980897  22.745961
2  16.006722  15.428583
3  24.242936  24.328510
4  38.693657  39.709190


In [678]:
print("Avg phi: ", avg_phi[0:10])
print("Avg diff: ", avg_diff[0:10])
print("Simple estimator: ", simple_estimator[0:10])

Avg phi:  0     3.518406
1    15.927225
2     8.721796
3    11.043561
4    22.703530
5     1.659202
6    -3.889531
7     1.539861
8     0.195419
9    11.944195
dtype: float64
Avg diff:  0    2.356845
1   -0.450762
2   -0.956002
3    0.130395
4    1.773729
5    0.359886
6   -0.604183
7    0.504040
8   -2.102164
9   -0.474768
dtype: float64
Simple estimator:  0    23.193561
1    23.482889
2    17.115027
3    24.089714
4    36.668732
5    26.592275
6    23.700632
7    28.501394
8    23.154597
9    22.462189
dtype: float64


In [679]:
# influence = avg_phi + avg_diff - simple_estimator
influence = avg_phi + avg_diff 

In [680]:
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 4.586+-0.325


In [681]:
means, stds = np.mean(influence, axis=0), sem(influence, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))

Estimation effect: 8.407+-0.814


In [577]:
data[['t', 'yf', 'ycf', 'p1', 'p0', 'predicted_t', 'predicted_y', 'phi_0', 'phi_1']]

,t,yf,ycf,p1,p0,predicted_t,predicted_y,phi_0,phi_1
0,1,24.534220,19.525653,0.283647,0.716353,0,24.537406,21.875898,24.537391
1,0,22.981254,26.063482,0.067996,0.932004,0,22.986151,22.986144,26.201720
2,0,16.006497,23.937630,0.107668,0.892332,0,16.004629,16.004631,20.788942
3,0,24.243270,23.527574,0.129877,0.870123,0,24.245512,24.245508,23.673081
4,0,38.693296,25.436166,0.092954,0.907046,0,38.689775,38.689780,28.794824
5,0,27.087900,25.415822,0.258861,0.741139,0,27.091902,27.091895,26.022017
6,0,22.757563,24.960663,0.372819,0.627181,0,22.758572,22.758570,24.331184
7,0,29.195772,26.382318,0.261353,0.738647,0,29.203032,29.203019,27.710891
8,0,20.112121,26.043065,0.310520,0.689480,0,20.073287,20.073362,25.880076
9,0,21.928347,23.248216,0.106283,0.893717,0,21.911732,21.911757,24.310108


In [482]:
data.t.value_counts()

0    608
1    139
Name: t, dtype: int64

## Sample method

In [628]:
def cal_influence_function(delta, data_train, data_test):
    features = cov.copy()
    features.append(treatment)

    ## Fit treatment
    model_t = RandomForestClassifier(n_estimators=10)
    model_t.fit(data_train[cov], data_train[treatment])
    data_test['predicted_t'] = model_t.predict(data_test[cov])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data_train[features], data_train[outcome])
    
    #data_train_treated = data_train[data_train[treatment] == 1]
    #data_train_control = data_train[data_train[treatment] == 0]
    
    #model_y_treated = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    #model_y_treated.fit(data_train_treated[cov], data_train_treated[outcome])
 
    #model_y_control = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    #model_y_control.fit(data_train_control[cov], data_train_control[outcome])
    data_test['predicted_y'] = model_y.predict(data_test[features])

    ### Performance
    print("------------- Performance in treatment model --------------------")
    print("F1 score: ", f1_score(data_test['t'], data_test['predicted_t'], average="macro"))
    print("Precision: ", precision_score(data_test['t'], data_test['predicted_t'], average="macro"))
    print("Recall: ",recall_score(data_test['t'], data_test['predicted_t'], average="macro")) 
    print("\n")
    # mean squared error
    mse = np.sum((data_test['yf'] - data_test['predicted_y'].values)**2)
    rmse = np.sqrt(mse/len(data_test))
    print("------------- Performance in outcome model --------------------")
    print("RMSE: ", rmse)
    
    ## Compute propensity score
    data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]
    data_test['p0'] = 1 - data_test['p1']
    
    data_test['predicted_y'] = 0

    #data_test_treated = data_test[data_test[treatment] == 1]
    #data_test_control = data_test[data_test[treatment] == 0]
    
    ## fit outcome
    #data_test.loc[data_test_treated.index.values,'predicted_y'] = model_y_treated.predict(data_test_treated[cov])
    #data_test.loc[data_test_control.index.values,'predicted_y'] = model_y_control.predict(data_test_control[cov])

    ## Compute counterfactual outcome with no treatment
    data_pos = data_test.copy()
    data_pos[treatment] = 1
    data_test['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data_test.copy()
    data_neg[treatment] = 0
    data_test['cf0'] = model_y.predict(data_neg[features])
    
    ## Compute incremental score 
    data_test['q1'] = (delta * data_test['p1']) / (delta * data_test['p1'] + data_test['p0'])
    data_test['q0'] = 1 - data_test['q1']
    estimator = data_test['q1']*data_test['cf1'] + data_test['q0']*data_test['cf0']

    # Y - mu(X,A)
    data_test['residual'] = data_test[outcome] - data_test['predicted_y']
    data_test['residual_cf0'] = data_test[outcome] - data_test['cf0']
    data_test['residual_cf1'] = data_test[outcome] - data_test['cf1']

    data_test['w0'] = [0 if data_test.loc[i,'predicted_t'] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
    data_test['w1'] = [0 if data_test.loc[i,'predicted_t'] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]


    data_test['phi_0'] = data_test['w0'] * data_test['residual_cf0'] + data_test['cf0'] 
    data_test['phi_1'] = data_test['w1'] * data_test['residual_cf1'] + data_test['cf1']

    avg_phi = -data_test['q1']*data_test['phi_1'] + data_test['q0']*data_test['phi_0']

    diff_cf = data_test['cf1'] - data_test['cf0']
    diff_ps = data_test[treatment] - data_test['p1']
    modi_ps = (delta*data_test['p1'] + data_test['p0'])**2

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps
    influence = avg_phi + avg_diff
    return influence

In [629]:
k = 2
num_of_delta = 5
delta_seq = np.linspace(0.5, 5, num_of_delta)

kf = KFold(n_splits = k, shuffle = True, random_state = 1)
effect = []
fold = 0

influence_sample = []
for train_index, test_index in tqdm(kf.split(data)):
    
    df_train = data.iloc[train_index]
    df_test = data.iloc[test_index]
    
    df_train = df_train.reset_index().drop(columns = ['index'])
    df_test = df_test.reset_index().drop(columns = ['index'])

    
    influence_delta = []
    
    for sample_delta in tqdm(delta_seq):
        influence = cal_influence_function(sample_delta, df_train, df_test)
        influence_delta.append(np.mean(influence))
        
    influence_sample.append(np.mean(influence_delta))

0it [00:00, ?it/s]
 20%|██        | 1/5 [00:02<00:09,  2.44s/it]

------------- Performance in treatment model --------------------
F1 score:  0.5272428794221456
Precision:  0.5567716141929036
Recall:  0.5286290322580645


------------- Performance in outcome model --------------------
RMSE:  1.224589853338812



 40%|████      | 2/5 [00:04<00:07,  2.44s/it]

------------- Performance in treatment model --------------------
F1 score:  0.48605492867787947
Precision:  0.5308970807585766
Recall:  0.5073084677419355


------------- Performance in outcome model --------------------
RMSE:  1.224589853338812



 60%|██████    | 3/5 [00:07<00:04,  2.46s/it]

...

------------- Performance in treatment model --------------------
F1 score:  0.44799154334038055
Precision:  0.44324460070316424
Recall:  0.4898881431767338


------------- Performance in outcome model --------------------
RMSE:  1.2806683842432256



 80%|████████  | 4/5 [00:09<00:02,  2.44s/it]

------------- Performance in treatment model --------------------
F1 score:  0.4755361751357124
Precision:  0.5508264462809918
Recall:  0.5082550335570469


------------- Performance in outcome model --------------------
RMSE:  1.2806683842432256



100%|██████████| 5/5 [00:12<00:00,  2.45s/it]
2it [00:24, 12.37s/it]

------------- Performance in treatment model --------------------
F1 score:  0.4755361751357124
Precision:  0.5508264462809918
Recall:  0.5082550335570469


------------- Performance in outcome model --------------------
RMSE:  1.2806683842432256


In [630]:
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 4.016+-0.031


In [631]:
means, stds = np.mean(influence_sample, axis=0), sem(influence_sample, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))

Estimation effect: 0.369+-0.525


## Predictive model

In [632]:
data = pd.read_csv(DATA_PATH, header = None, names = col)

In [248]:
data.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [249]:
data.t.value_counts()

0    608
1    139
Name: t, dtype: int64

In [221]:
con_feature = ["x" + str(a) for a in range(1,7)]
cat_feature = ["x" + str(a) for a in range(7, 26)]

In [222]:
scaler = StandardScaler()
data[con_feature] = scaler.fit_transform(data[con_feature])

In [223]:
data.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.931652,3.500591,2.253801,7.136441,-0.528957,-0.343685,1.129310,0.161811,-0.316815,1.296084,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,2.175966,5.952101,1.257592,6.553022,-1.738109,-1.803210,0.384085,2.245823,-0.629611,1.296084,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,2.180294,7.175734,2.384100,7.192645,-0.807992,-0.203082,-0.361140,-0.880195,0.809248,-0.526908,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,3.587662,7.787537,4.009365,7.712456,0.390344,0.596982,-1.851590,-0.880195,-0.004020,-0.858362,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,2.372618,5.461871,2.481631,7.232739,-1.045929,-0.603114,0.011473,0.161811,0.684130,-0.361182,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [224]:
def performance(data, model_y, model_t, k):
    f1 = []
    precision_score_ = []
    rmse = []
    kf = KFold(n_splits = k, shuffle = True, random_state = 1)
    
    for train_index, test_index in tqdm(kf.split(data)):
        
        df_train = data.iloc[train_index]
        df_test = data.iloc[test_index]

        df_train = df_train.reset_index().drop(columns = ['index'])
        df_test = df_test.reset_index().drop(columns = ['index'])
        
        model_t.fit(df_train[cov].values, df_train[treatment].values)
        
        df_test['predicted_t'] = model_t.predict(df_test[cov])
        
        model_y.fit(df_train[features].values, df_train[outcome].values)
        df_test['predicted_y'] = model_y.predict(df_test[features])

        precision_score_sample = precision_score(df_test['t'], df_test['predicted_t'], average="macro")
        mse = np.sum((df_test['yf'] - df_test['predicted_y'].values)**2)
        rmse_sample = np.sqrt(mse/len(df_test))
        
        precision_score_.append(precision_score_sample)
        rmse.append(rmse_sample)
        
    return np.mean(precision_score_), np.mean(rmse)

In [225]:
# cost function for base models
def rmse(yreal, yhat):
    return sqrt(mean_squared_error(yreal, yhat))
def get_super_learner_outcome(X):
    ensemble = SuperLearner(scorer=rmse, folds=10, shuffle=True, sample_size=len(X))
    
    models = list()
    models.append(LinearRegression())
    models.append(ElasticNet())
    models.append(SVR(gamma='scale'))
    models.append(DecisionTreeRegressor())
    models.append(KNeighborsRegressor())
    models.append(AdaBoostRegressor())
    models.append(BaggingRegressor(n_estimators=100))
    models.append(RandomForestRegressor(n_estimators=100))
    models.append(ExtraTreesRegressor(n_estimators=100))
    ensemble.add(models)
    ensemble.add_meta(LinearRegression())
    return ensemble

def f1(y, p): return f1_score(y, p, average='micro')


def get_super_learner_treatment(X):
    ensemble = SuperLearner(scorer=f1, random_state=1)
    
    models = list()
    models.append(KNeighborsClassifier(3))
    models.append(RandomForestClassifier(n_estimators=30, random_state=1))
    models.append(GradientBoostingClassifier())
    models.append(SVC())
    models.append(AdaBoostClassifier())
    models.append(MLPClassifier(alpha=1, max_iter=1000))
    models.append(GaussianProcessClassifier(1.0 * RBF(1.0)))
    

    ensemble.add(models)
    ensemble.add_meta(LogisticRegression())
    return ensemble



In [226]:
model_y = get_super_learner_outcome(data)
model_t = get_super_learner_treatment(data)

pre, rmse = performance(data, model_y, model_t, 5)
print("Precision: {}, RMSE: {}".format(pre, rmse))

0it [00:00, ?it/s]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [00:26, 26.55s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [00:46, 24.44s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

Precision: 0.40692170022371366, RMSE: 1.085390649212887


In [227]:
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_t = RandomForestClassifier(n_estimators=10)

pre, rmse = performance(data, model_y, model_t, 5)
print("Precision: {}, RMSE: {}".format(pre, rmse))

5it [00:18,  3.67s/it]

Precision: 0.53269385720717, RMSE: 1.23093932969195


In [228]:
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_t = GradientBoostingClassifier(n_estimators=10)

pre, rmse = performance(data, model_y, model_t, 5)
print("Precision: {}, RMSE: {}".format(pre, rmse))

1it [00:03,  3.67s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [00:07,  3.66s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [00:10,  3.65s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

Precision: 0.40651974121772777, RMSE: 1.23093932969195


## Sample method with super learners

In [285]:
def cal_influence_function(delta, data_train, data_test):
    features = cov.copy()
    features.append(treatment)

    ## Fit treatment
    model_t = RandomForestClassifier(n_estimators=10)
    model_t.fit(data_train[cov], data_train[treatment])
    data_test['predicted_t'] = model_t.predict(data_test[cov])

    ## Fit outcome
    #model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y  = get_super_learner_outcome(data_train)
    model_y.fit(data_train[features].values, data_train[outcome].values)
    
    #data_train_treated = data_train[data_train[treatment] == 1]
    #data_train_control = data_train[data_train[treatment] == 0]
    
    #model_y_treated = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    #model_y_treated.fit(data_train_treated[cov], data_train_treated[outcome])
 
    #model_y_control = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    #model_y_control.fit(data_train_control[cov], data_train_control[outcome])
    data_test['predicted_y'] = model_y.predict(data_test[features])

    ### Performance
    print("------------- Performance in treatment model --------------------")
    print("F1 score: ", f1_score(data_test['t'], data_test['predicted_t'], average="macro"))
    print("Precision: ", precision_score(data_test['t'], data_test['predicted_t'], average="macro"))
    print("Recall: ",recall_score(data_test['t'], data_test['predicted_t'], average="macro")) 
    print("\n")
    # mean squared error
    mse = np.sum((data_test['yf'] - data_test['predicted_y'].values)**2)
    rmse = np.sqrt(mse/len(data_test))
    print("------------- Performance in outcome model --------------------")
    print("RMSE: ", rmse)
    
    ## Compute propensity score
    data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]
    data_test['p0'] = 1 - data_test['p1']
    
    data_test['predicted_y'] = 0

    #data_test_treated = data_test[data_test[treatment] == 1]
    #data_test_control = data_test[data_test[treatment] == 0]
    
    ## fit outcome
    #data_test.loc[data_test_treated.index.values,'predicted_y'] = model_y_treated.predict(data_test_treated[cov])
    #data_test.loc[data_test_control.index.values,'predicted_y'] = model_y_control.predict(data_test_control[cov])

    ## Compute counterfactual outcome with no treatment
    data_pos = data_test.copy()
    data_pos[treatment] = 1
    data_test['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data_test.copy()
    data_neg[treatment] = 0
    data_test['cf0'] = model_y.predict(data_neg[features])
    
    ## Compute incremental score 
    data_test['q1'] = (delta * data_test['p1']) / (delta * data_test['p1'] + data_test['p0'])
    data_test['q0'] = 1 - data_test['q1']
    estimator = data_test['q1']*data_test['cf1'] + data_test['q0']*data_test['cf0']

    # Y - mu(X,A)
    data_test['residual'] = data_test[outcome] - data_test['predicted_y']
    data_test['residual_cf0'] = data_test[outcome] - data_test['cf0']
    data_test['residual_cf1'] = data_test[outcome] - data_test['cf1']

    data_test['w0'] = [0 if data_test.loc[i,'predicted_t'] == 1 else 1/data_test.loc[i, 'p0'] for i in range(len(data_test))]
    data_test['w1'] = [0 if data_test.loc[i,'predicted_t'] == 0 else 1/data_test.loc[i, 'p1'] for i in range(len(data_test))]


    data_test['phi_0'] = data_test['w0'] * data_test['residual_cf0'] + data_test['cf0'] 
    data_test['phi_1'] = data_test['w1'] * data_test['residual_cf1'] + data_test['cf1']

    avg_phi = data_test['q1']*data_test['phi_1'] + data_test['q0']*data_test['phi_0']

    diff_cf = data_test['cf1'] - data_test['cf0']
    diff_ps = data_test[treatment] - data_test['p1']
    modi_ps = (delta*data_test['p1'] + data_test['p0'])**2

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps

    avg_diff = (delta*diff_cf*diff_ps) / modi_ps
    influence = avg_phi + avg_diff
    return influence

In [286]:
k = 2
num_of_delta = 5
delta_seq = np.linspace(0.5, 5, num_of_delta)

kf = KFold(n_splits = k, shuffle = True, random_state = 1)
effect = []
fold = 0

influence_sample = []
for train_index, test_index in tqdm(kf.split(data)):
    
    df_train = data.iloc[train_index]
    df_test = data.iloc[test_index]
    
    df_train = df_train.reset_index().drop(columns = ['index'])
    df_test = df_test.reset_index().drop(columns = ['index'])

    
    influence_delta = []
    
    for sample_delta in tqdm(delta_seq):
        influence = cal_influence_function(sample_delta, df_train, df_test)
        influence_delta.append(np.mean(influence))
        
    influence_sample.append(np.mean(influence_delta))

0it [00:00, ?it/s]
  0%|          | 0/5 [00:00<?, ?it/s]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.2] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.10] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.9] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostreg

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [baggingregressor.0.5] Could not score baggingregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.1] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.4] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [kneighborsregressor.0.1] Could not score kneighborsregressor. 

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.6] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.8] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.7] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [svr.0.1] Could not score svr. Details:
TypeError("'numpy

------------- Performance in treatment model --------------------
F1 score:  0.4762861491628615
Precision:  0.5261796042617961
Recall:  0.5043346774193549


------------- Performance in outcome model --------------------
RMSE:  1.247753175536427



 20%|██        | 1/5 [00:08<00:32,  8.03s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.4] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.9] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.8] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.2]

...


 80%|████████  | 4/5 [00:32<00:08,  8.05s/it]/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.6] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.4] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.1] Could not score adaboostregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [adaboostregressor.0.5]

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [baggingregressor.0.2] Could not score baggingregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [baggingregressor.0.5] Could not score baggingregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [baggingregressor.0.9] Could not score baggingregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [kneighborsregressor.0.1] Could not score kneighborsregressor. Details:
Typ

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [linearregression.0.7] Could not score linearregression. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.7] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.8] Could not score extratreesregressor. Details:
TypeError("'numpy.float64' object is not callable")
  (name, inst_name, exc), MetricWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [extratreesregressor.0.5] Could not score extratreesregressor. 

------------- Performance in treatment model --------------------
F1 score:  0.4611346018322763
Precision:  0.49008036162732294
Recall:  0.4982326621923937


------------- Performance in outcome model --------------------
RMSE:  1.2874937768462067



100%|██████████| 5/5 [00:40<00:00,  8.04s/it]
2it [01:19, 39.87s/it]


In [287]:
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 3.854+-0.056


In [288]:
means, stds = np.mean(influence_sample, axis=0), sem(influence_sample, axis=0)
print('Estimation effect: {:.3f}+-{:.3f}'.format(means, stds))

Estimation effect: 6.517+-0.252


## Dowhy package 

In [633]:
data.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [634]:
data.p1

AttributeError: 'DataFrame' object has no attribute 'p1'

In [635]:
# Create a causal model from the data and given graph.
model = CausalModel(data=data,
                    treatment=treatment,
                    outcome=outcome,
                    common_causes=cov)

data[treatment] = [bool(x) for x in data[treatment]]
# Identify causal effect and return target estimands
identified_estimand = model.identify_effect()

# Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(
    identified_estimand, method_name="backdoor.propensity_score_weighting")

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['t'] on outcome ['yf']
INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x5', 'x3', 'x8', 'x6', 'x12', 'x9', 'x24', 'x19', 'x25', 'x21', 'x16', 'x10', 'x20', 'x4', 'x18', 'x17', 'U', 'x15', 'x22', 'x11', 'x2', 'x1', 'x23', 'x14', 'x7', 'x13']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_estimator:INFO: Using Propensity Score Weighting Estimator
INFO:dowhy.causal_estimator:b: yf~t+x5+x3+x8+x6+x12+x9+x24+x19+x25+x21+x16+x10+x20+x4+x18+x17+x15+x22+x11+x2+x1+x23+x14+x7+x13
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [636]:
print("Causal Estimate is " + str(estimate.value))


Causal Estimate is 3.4097378244105645


In [637]:
true_effect = data['mu1'] - data['mu0']
means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
print('True effect: {:.3f}+-{:.3f}'.format(means, stds))

True effect: 4.016+-0.031


In [644]:
data[['t', 'p1']].head()

,t,p1
0,True,0.283647
1,False,0.067996
2,False,0.107668
3,False,0.129877
4,False,0.092954


In [645]:
data['ips_weight'] = (1 / len(data)) * (
    data['t'] / data['p1'] +
    (1 - data[treatment]) / (1 - data['p1']))

# Calculating the effect
data['d_y'] = (data['ips_weight'] *
                     data['t'] *
                     data[outcome])
data['dbar_y'] = (data['ips_weight'] *
                        (1 - data['t']) *
                        data[outcome])
est = data['d_y'].sum() - data['dbar_y'].sum()
est

In [646]:
data['ips_weight'] = (1 / len(data)) * (
    data[treatment] / data['p1'] +
    (1 - data[treatment]) / (1 - data['p1']))

# Calculating the effect
data['d_y'] = (data['ips_weight'] *
                     data[treatment] *
                     data[outcome])
data['dbar_y'] = (data['ips_weight'] *
                        (1 - data[treatment]) *
                        data[outcome])
est = data['d_y'].sum() - data['dbar_y'].sum()

In [647]:
est

In [491]:
data[['d_y', 'dbar_y', 'ips_weight']]

,d_y,dbar_y,ips_weight
0,0.000000,0.000000,0.000000
1,0.000000,0.031075,0.001352
2,0.000000,0.021644,0.001352
3,0.000000,0.032782,0.001352
4,0.000000,0.052321,0.001352
5,0.000000,0.040291,0.001487
6,0.000000,0.033850,0.001487
7,0.000000,0.039479,0.001352
8,0.000000,0.027196,0.001352
9,0.000000,0.032617,0.001487


In [492]:
est

In [493]:
# A1, A0 = ps_gbm, 1 - ps_gbm

w1, w0 = 1 / data['p1'], 1 / data['p0']

mu1 = np.sum(data[treatment]  * data['t'] * w1) / np.sum(data['t']  * w1)
mu0 = np.sum(data[treatment] * (1 - data['t'] ) * w0) / np.sum((1 - data['t'] ) * w0)

TE_gbm_ps = mu1 - mu0
TE_gbm_ps

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/pandas/core/computation/expressions.py:178: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [494]:
num_units = len(data)
p_treatment = sum(data[treatment]) / num_units
data['ips_stabilized_weight'] = (1 / num_units) * (
    data[treatment] / data['p1'] * p_treatment + (1 - data[treatment]) /
    (1 - data['p1']) * (1 - p_treatment))

# Calculating the effect
data['d_y'] = (data['ips_stabilized_weight'] *
                     data[treatment] *
                     data[outcome])
data['dbar_y'] = (data['ips_stabilized_weight'] *
                        (1 - data[treatment]) *
                        data[outcome])
est = data['d_y'].sum() - data['dbar_y'].sum()
est

In [558]:
data['w0'] = [0 if data.loc[i,'predicted_t'] == 1 else 1/data.loc[i, 'p0'] for i in range(len(data))]
data['w1'] = [0 if data.loc[i,'predicted_t'] == 0 else 1/data.loc[i, 'p1'] for i in range(len(data))]

data['y_w0'] = data['w0']*data['yf']
data['y_w1'] = data['w1']*data['yf']

In [559]:
np.mean(data[treatment]*data[outcome] / data['p1'] - (1 - data[treatment])*data[outcome]/data['p0'])

In [560]:
data.head()

,t,yf,ycf,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,predicted_y,predicted_t,p1,p0,ips_weight,d_y,dbar_y,cf1,cf0,q1,q0,residual,residual_cf0,residual_cf1,w0,w1,phi_0,phi_1,a1,a0,y_w0,y_w1
0,1,24.534220,19.525653,21.221590,24.830999,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,24.537406,0,0.283647,0.716353,0.004720,0.115791,0.000000,24.537406,21.863292,0.497462,0.502538,-0.003185,2.670929,-0.003185,1.395959,0.0,25.591799,24.537406,2.662884,-1.054393,34.248772,0.0
1,0,22.981254,26.063482,21.354089,24.837223,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,22.986151,0,0.067996,0.932004,0.001436,0.000000,0.033009,26.206352,22.986151,0.154257,0.845743,-0.004898,-0.004898,-3.225098,1.072957,0.0,22.980897,26.206352,-44.210534,3.225455,24.657891,0.0
2,0,16.006497,23.937630,18.040368,24.668592,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0,16.004629,0,0.107668,0.892332,0.001500,0.000000,0.024013,20.796127,16.004629,0.231743,0.768257,0.001868,0.001868,-4.789630,1.120659,0.0,16.006722,20.796127,-39.693588,4.789405,17.937831,0.0
3,0,24.243270,23.527574,25.759277,25.024775,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0,24.245512,0,0.129877,0.870123,0.001539,0.000000,0.037298,23.672203,24.245512,0.271752,0.728248,-0.002241,-0.002241,0.571068,1.149263,0.0,24.242936,23.672203,3.823668,-0.570733,27.861900,0.0
4,0,38.693296,25.436166,38.671451,25.431081,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,38.689775,0,0.092954,0.907046,0.001476,0.000000,0.057107,28.780193,38.689775,0.203948,0.796052,0.003522,0.003522,9.913103,1.102480,0.0,38.693657,28.780193,96.735669,-9.913464,42.658582,0.0


In [561]:
data['a1'] = data[outcome] / data['p1'] - data['p0']*data['cf1'] / data['p1'] - data['cf0']
data['a0'] = -(data[outcome] / data['p0'] - data['p1']*data['cf0'] / data['p0']) + data['cf1']

In [564]:
# np.mean(data['a1'] + data['a0'])

In [565]:
np.mean(np.sum(data['a1']) + np.sum(data['a0']))

In [521]:
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_{}.csv".format(10)
data = pd.read_csv(DATA_PATH, header = None, names = col)

In [522]:
## Fit treatment
# model_t = LogisticRegression()
model_t = RandomForestClassifier(n_estimators=10)

model_t.fit(data[cov], data[treatment])

data['p1'] = model_t.predict_proba(data[cov])[:,1]

In [542]:
data['ips_weight'] = (1 / len(data)) * (
    data['t'] / data['p1'] +
    (1 - data[treatment]) / (1 - data['p1']))

# Calculating the effect
data['d_y'] = (data['ips_weight'] *
                     data['t'] *
                     data[outcome])
data['dbar_y'] = (data['ips_weight'] *
                        (1 - data['t']) *
                        data[outcome])
est = data['d_y'].sum() - data['dbar_y'].sum()

In [543]:
est

In [525]:
data.p1

0      0.6
1      0.0
2      0.2
3      0.0
4      0.1
5      0.0
6      0.1
7      0.2
8      0.1
9      0.1
10     0.1
11     0.1
12     0.0
13     0.2
14     0.1
15     0.0
16     0.1
17     0.1
18     0.8
19     0.1
20     0.1
21     0.0
22     0.9
23     0.0
24     0.0
25     0.0
26     0.7
27     0.2
28     0.0
29     0.1
30     0.1
31     0.1
32     0.0
33     0.1
34     0.0
35     0.1
36     0.2
37     0.0
38     0.1
39     0.2
40     0.2
41     0.7
42     0.7
43     0.8
44     0.7
45     0.0
46     0.1
47     0.1
48     0.1
49     0.6
50     1.0
51     0.0
52     0.1
53     0.9
54     0.0
55     0.0
56     0.1
57     0.1
58     0.6
59     0.9
60     0.0
61     0.0
62     0.0
63     0.8
64     0.0
65     0.0
66     0.0
67     0.1
68     0.1
69     0.1
70     0.0
71     0.1
72     0.0
73     0.2
74     0.7
75     0.8
76     0.7
77     0.0
78     0.1
79     0.8
80     0.3
81     0.0
82     0.1
83     0.2
84     0.0
85     0.7
86     0.0
87     0.9
88     0.1
89     0.1
90     0.1